# Clinical trials: ML Classification


In [1]:
import time
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
import sys
import numpy as np

In [2]:
base_color = "#3298D0"
plot_size = (14, 10)

# sys.stdout = open('/dev/stdout', 'w')

In [3]:
# Path for csv folder & file
path_to_csv_file = os.path.abspath('../data/csv/')

# json file
csv_file = '/clean_data'

## Import csv file

In [4]:
# Import json into a dataframe
clean_csv_file = '{}{}.csv'.format(path_to_csv_file, csv_file)

In [5]:
'''Use dask to improve data loading
https://www.kaggle.com/shikhar1/yet-another-pandas-tutorial'''

# breaks with large json file
df = pd.read_csv(clean_csv_file)

In [6]:
df.shape

(10000, 10)

In [7]:
# remove Unnamed column
df = df.drop(columns = ['Unnamed: 0'])
df = df.drop(columns = ['original_date'])

In [8]:
df.columns

Index(['id', 'source', 'brief_title', 'condition', 'full_description',
       'summary', 'full_date', 'year'],
      dtype='object')

## Optimize memory usage

In [9]:
print(df.memory_usage(deep=True))
df.info(verbose=True)

Index                     80
id                    680000
source                854529
brief_title          1447701
condition             766621
full_description    15993235
summary              7424117
full_date             670000
year                   80000
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
id                  10000 non-null object
source              10000 non-null object
brief_title         10000 non-null object
condition           10000 non-null object
full_description    10000 non-null object
summary             10000 non-null object
full_date           10000 non-null object
year                10000 non-null int64
dtypes: int64(1), object(7)
memory usage: 625.1+ KB


In [10]:
df.dtypes

id                  object
source              object
brief_title         object
condition           object
full_description    object
summary             object
full_date           object
year                 int64
dtype: object

In [11]:
# Change data types
df['id'] = df['id'].astype('str')
for col in ['source', 'condition']:
    df[col] = df[col].astype('category')

In [12]:
df['full_date'] =  pd.to_datetime(df['full_date'])

In [13]:
print(df.memory_usage(deep=True))
df.info(verbose=True)

Index                     80
id                    680000
source                468036
brief_title          1447701
condition             537645
full_description    15993235
summary              7424117
full_date              80000
year                   80000
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
id                  10000 non-null object
source              10000 non-null category
brief_title         10000 non-null object
condition           10000 non-null category
full_description    10000 non-null object
summary             10000 non-null object
full_date           10000 non-null datetime64[ns]
year                10000 non-null int64
dtypes: category(2), datetime64[ns](1), int64(1), object(4)
memory usage: 887.2+ KB


In [14]:
print(df.shape)
print(df.nunique())

(10000, 8)
id                  10000
source               3222
brief_title          9980
condition            4371
full_description     6469
summary              9974
full_date            3675
year                   20
dtype: int64


## Test: Classification through a list of terms
Filtering by list of words related to cancer research. 

In [120]:
# 1st approach: use a list of terms as reference
cancer_words = ['cancer', 'oncology', 'melanoma', 'sarcoma']
pat = '|'.join([r'\b{}\b'.format(cw) for cw in cancer_words])

In [16]:
df['cancer'] = df['summary'].str.contains(pat, case = False).astype(int)

In [17]:
# Create a new dataset for records with cancer terms
df_cancer = df[df['cancer'] == 1]

In [18]:
df_cancer.nunique()

id                  1273
source               555
brief_title         1273
condition            538
full_description     900
summary             1273
full_date            989
year                  20
cancer                 1
dtype: int64

In [19]:
df_cancer['condition'].unique()

[Cardiovascular Diseases, Breast Cancer, Colorectal Cancer, Drug/Agent Toxicity by Tissue/Organ, Brain and Central Nervous System Tumors, ..., CRE-induced SBO, Clinical Stage III Gastric Cancer AJCC v8, Hematologic Malignancy, Shoulder Dysfunction, Chronic Phase Chronic Myelogenous Leukemia, BC...]
Length: 538
Categories (538, object): [Cardiovascular Diseases, Breast Cancer, Colorectal Cancer, Drug/Agent Toxicity by Tissue/Organ, ..., Clinical Stage III Gastric Cancer AJCC v8, Hematologic Malignancy, Shoulder Dysfunction, Chronic Phase Chronic Myelogenous Leukemia, BC...]

In [20]:
# Registries across diseases: Cancer & Cardiovascular disease
df_cancer.loc[(df_cancer['condition'] == 'Cardiovascular Diseases') & (df_cancer['cancer'] == 1)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 8 to 8
Data columns (total 9 columns):
id                  1 non-null object
source              1 non-null category
brief_title         1 non-null object
condition           1 non-null category
full_description    1 non-null object
summary             1 non-null object
full_date           1 non-null datetime64[ns]
year                1 non-null int64
cancer              1 non-null int64
dtypes: category(2), datetime64[ns](1), int64(2), object(4)
memory usage: 379.4+ KB


## ML Pipeline

### DF transformations
- Check which ones can be added to the preprocessor
- Info: http://www.ultravioletanalytics.com/blog/tf-idf-basics-with-pandas-scikit-learn

In [21]:
# Create a column combining title + full description
df['text'] = df['brief_title'] + df['summary'] + df['full_description']

In [22]:
# Clean extra blank spaces
df.text = df.text.replace('\s+', ' ', regex=True)

### Create a vocabulary

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
import re

In [103]:
pat_numbers = lambda x: re.sub(r'(\d)+', '', x.lower())

cv = CountVectorizer(stop_words='english',
                     preprocessor = pat_numbers,
                     max_features = 3000,
                     lowercase = True,
                     max_df = 0.5,
                     ngram_range = (1, 3))

In [104]:
docs = list(df['full_description'])

X = cv.fit_transform(docs)

In [105]:
len(cv.vocabulary_)

3000

In [106]:
cv.vocabulary_

{'develop': 741,
 'medication': 1582,
 'treatment': 2814,
 'using': 2884,
 'medical': 1577,
 'human': 1212,
 'laboratory': 1435,
 'setting': 2473,
 'characterize': 411,
 'population': 2011,
 'variety': 2909,
 'psychological': 2158,
 'measures': 1569,
 'time': 2768,
 'withdrawal': 2976,
 'post': 2019,
 'administration': 68,
 'determine': 734,
 'effects': 862,
 'subjective': 2660,
 'physiological': 1983,
 'challenge': 404,
 'chronic': 431,
 'evaluate': 942,
 'clinical': 448,
 'safety': 2396,
 'issues': 1407,
 'interaction': 1353,
 'dependent': 715,
 'clearly': 446,
 'following': 1058,
 'cessation': 402,
 'number': 1748,
 'drugs': 838,
 'alcohol': 115,
 'unclear': 2854,
 'syndrome': 2710,
 'follows': 1059,
 'model': 1645,
 'provide': 2148,
 'safe': 2394,
 'systematic': 2713,
 'method': 1607,
 'testing': 2749,
 'efficacy': 863,
 'behavioral': 276,
 'pharmacological': 1967,
 'treatments': 2828,
 'symptoms': 2709,
 'smoking': 2523,
 'study': 2617,
 'investigated': 1383,
 'acute': 48,
 'hrs':

In [107]:
# Check for values in dictionary
if 'breast cancer' in list(cv.vocabulary_):
  print('yes')
else:
  print('no')

yes


In [108]:
pd.DataFrame(X.toarray(), columns=cv.get_feature_names()).head()

,abdomen,abdominal,ability,ablation,able,abnormal,abnormalities,absence,absence disease,absence disease progression,...,written informed consent,year,year months,years,years age,years months,years old,young,youth,µg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### TF-IDF Model

In [109]:
from sklearn.feature_extraction.text import TfidfTransformer

In [110]:
transformer = TfidfTransformer()
tweights = transformer.fit_transform(X)
tweights

<10000x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 529882 stored elements in Compressed Sparse Row format>

In [111]:
# Pending: Reduced dimensionality



In [112]:
# turn weights data into a dataframe
tf = pd.DataFrame(tweights.toarray(), columns=cv.get_feature_names())

In [113]:
# Top terms by average tf-idf weight
weights = np.asarray(tweights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cv.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(5)

,term,weight
1890,patients,0.040086
2617,study,0.035337
2814,treatment,0.025002
2661,subjects,0.014800
810,dose,0.014793


In [123]:
# Check word
def check_weight(word):
    for w in word:
        try:
            print('{}: {}'.format(w, tf[w].mean()))
        except KeyError:
            print('{}: None'.format(w))

search_terms = ['cancer', 'breast cancer', 'migraine', 'sarcoma']
check_weight(search_terms)

cancer: 0.00926551970723474
breast cancer: 0.0016709429549899139
migraine: None
sarcoma: None


In [140]:
# Merge df y tf-idf data by index
pd.set_option('display.max_columns', None) 
tf.sample(5)
# pd.merge(df, tf, left_index=True, right_index=True).sample(1)

abdomen  abdominal  ability  ablation  able  abnormal  abnormalities  \
1569  0.0      0.0        0.0      0.0       0.0   0.0       0.0             
6661  0.0      0.0        0.0      0.0       0.0   0.0       0.0             
412   0.0      0.0        0.0      0.0       0.0   0.0       0.0             
2911  0.0      0.0        0.0      0.0       0.0   0.0       0.0             
7388  0.0      0.0        0.0      0.0       0.0   0.0       0.0             

      absence  absence disease  absence disease progression  absorption  \
1569  0.0      0.0              0.0                          0.0          
6661  0.0      0.0              0.0                          0.0          
412   0.0      0.0              0.0                          0.0          
2911  0.0      0.0              0.0                          0.0          
7388  0.0      0.0              0.0                          0.0          

      abstinence  abuse   ac  academic  acceptability  acceptable  accepted  \
1569  0.0         0.0    0.0  0.0       0.0            0.0         0.0        
6661  0.0         0.0    0.0  0.0       0.0            0.0         0.0        
412   0.0         0.0    0.0  0.0       0.0            0.0         0.0        
2911  0.0         0.0    0.0  0.0       0.0            0.0         0.0        
7388  0.0         0.0    0.0  0.0       0.0            0.0         0.0        

      access  accordance  according  account  accrual  accrual total  \
1569  0.0     0.0         0.0        0.0      0.0      0.0             
6661  0.0     0.0         0.0        0.0      0.0      0.0             
412   0.0     0.0         0.0        0.0      0.0      0.0             
2911  0.0     0.0         0.0        0.0      0.0      0.0             
7388  0.0     0.0         0.0        0.0      0.0      0.0             

      accrual total patients  accrued  accrued study  accuracy  accurate  \
1569  0.0                     0.0      0.0            0.0       0.000000   
6661  0.0                     0.0      0.0            0.0       0.000000   
412   0.0                     0.0      0.0            0.0       0.000000   
2911  0.0                     0.0      0.0            0.0       0.240846   
7388  0.0                     0.0      0.0            0.0       0.000000   

      acetaminophen  achieve  achieved  achieving  acid  acids  acquired  \
1569  0.0            0.0      0.0       0.0        0.0   0.0    0.0        
6661  0.0            0.0      0.0       0.0        0.0   0.0    0.0        
412   0.0            0.0      0.0       0.0        0.0   0.0    0.0        
2911  0.0            0.0      0.0       0.0        0.0   0.0    0.0        
7388  0.0            0.0      0.0       0.0        0.0   0.0    0.0        

      acquisition  act  acting  action  activated  activation    active  \
1569  0.0          0.0  0.0     0.0     0.0        0.0         0.000000   
6661  0.0          0.0  0.0     0.0     0.0        0.0         0.000000   
412   0.0          0.0  0.0     0.0     0.0        0.0         0.000000   
2911  0.0          0.0  0.0     0.0     0.0        0.0         0.000000   
7388  0.0          0.0  0.0     0.0     0.0        0.0         0.275597   

      activities  activity  actual  acuity  acupuncture  acute   ad  adapted  \
1569  0.0         0.000000  0.0     0.0     0.0          0.0    0.0  0.0       
6661  0.0         0.000000  0.0     0.0     0.0          0.0    0.0  0.0       
412   0.0         0.000000  0.0     0.0     0.0          0.0    0.0  0.0       
2911  0.0         0.000000  0.0     0.0     0.0          0.0    0.0  0.0       
7388  0.0         0.081247  0.0     0.0     0.0          0.0    0.0  0.0       

      adaptive  add  added  addition  additional  additionally  address  \
1569  0.0       0.0  0.0    0.000000  0.0         0.0           0.0       
6661  0.0       0.0  0.0    0.062377  0.0         0.0           0.0       
412   0.0       0.0  0.0    0.000000  0.0         0.0           0.0       
2911  0.0       0.0  0.0    0.00

### Topic Modeling
- https://nlpforhackers.io/topic-modeling/

### Test: Using Tensorflow 
- https://github.com/open-source-for-science/TensorFlow-Course?utm_campaign=explore-email&utm_medium=email&utm_source=newsletter&utm_term=weekly